In [2]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [50]:
from pydantic import BaseModel, Field, ValidationError
from langchain_core.output_parsers import PydanticOutputParser

class MeetingSummary(BaseModel):
    date: str = Field(description="회의 날짜")
    attendees: list[str] = Field(description="회의 참석자 명단", min_items=1)
    topics: list[str] = Field(description="회의에서 다룬 주요 안건", min_items=1)
    decisions: list[str] = Field(description="회의에서 결정된 사항", min_items=1)
    next_steps: list[str] = Field(description="다음 단계 또는 후속 조치", min_items=1)

parser = PydanticOutputParser(pydantic_object=MeetingSummary)
parser

PydanticOutputParser(pydantic_object=<class '__main__.MeetingSummary'>)

In [51]:
fmt = parser.get_format_instructions()
fmt

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"date": {"description": "회의 날짜", "title": "Date", "type": "string"}, "attendees": {"description": "회의 참석자 명단", "items": {"type": "string"}, "minItems": 1, "title": "Attendees", "type": "array"}, "topics": {"description": "회의에서 다룬 주요 안건", "items": {"type": "string"}, "minItems": 1, "title": "Topics", "type": "array"}, "decisions": {"description": "회의에서 결정된 사항", "items": {"type": "string"}, "minItems": 1, "title": "Decisions", "type": "array"}, "next_steps": {"description": "다음 단계 또는 후속 조치", "items": {"type": "string"}, "

In [49]:
system_prompt = """
당신은 회의록을 전문적으로 요약하는 15년차 베테랑
내용을 직관적으로 보이게 요약해주세요.

1. 핵심 내용만 추출해주세요.
2. 요약 내용이 너무 길지 않게 해주세요
3. 다음 회의 내용이 무엇일지 예측해서 어떤 것을 준비해야할지 알려주세요

[양식]
{fmt}
"""

user_prompt = """
회의 내용에 대해서 한눈에 알아볼 수 있게 요약해주세요

[요약]
{text}
"""
prompt = ChatPromptTemplate.from_messages({
    ("system",system_prompt),
    ("user", user_prompt)
}).partial(fmt=fmt)

In [52]:
prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"date": {"description": "회의 날짜", "title": "Date", "type": "string"}, "attendees": {"description": "회의 참석자 명단", "items": {"type": "string"}, "minItems": 1, "title": "Attendees", "type": "array"}, "topics": {"description": "회의에서 다룬 주요 안건", "items": {"type": "string"}, "minItems": 1, "title": "Topics", "type": "array"}, "decisions": {"description": "회의에서 결정된 사항", "items": {"type": "string"}, "minItems": 1, "title": "Decisions", "type": "

In [53]:
chain = prompt | model | parser
text = """
개발팀 박민수 팀장은 신제품 개발 진행 상황을 먼저 보고했다. 현재 전체 개발 일정의 약 80%가 완료된 상태이며, 남은 작업은 주로 UI/UX 개선과 안정성 테스트라고 설명했다. 특히 지난주 사내 테스트에서 발견된 버그 중 일부는 이미 수정되었으며, 나머지는 이번 주 내에 해결될 예정이라고 덧붙였다. 그는 10월 중순까지 베타 테스트를 진행한 뒤, 테스트 결과를 반영하여 11월 초에 정식 출시가 가능하다고 자신 있게 말했다.

이에 대해 마케팅팀 최지현 팀장은 출시 전후 마케팅 전략을 발표했다. 주요 타겟 고객층은 20~30대 직장인으로 설정했으며, 이들이 자주 사용하는 인스타그램과 틱톡을 중심으로 온라인 홍보를 강화하기로 했다. 특히 짧은 영상 콘텐츠와 사용 후기 기반 광고를 집중 제작하여 소비자들의 호감을 끌어낼 계획이다. 또한 제품 출시 전인 10월 15일부터 사전 예약 이벤트를 열어 초기 고객을 확보하고, 예약 고객에게는 소정의 사은품을 제공하는 방안을 제안했다.

기획팀 이영희 팀장은 예산과 관련된 사항을 보고했다. 초기 마케팅 예산으로 5천만 원을 배정했으며, 이 금액은 SNS 광고, 이벤트 진행, 프로모션 영상 제작 등에 사용될 예정이다. 만약 광고 성과가 예상보다 좋아 추가 홍보가 필요할 경우, 별도의 예산 승인을 받아 집행할 수 있도록 계획을 세웠다고 설명했다.

대표 김철수 사장은 참석자들의 보고를 듣고 몇 가지 중요한 사항을 강조했다. 첫째, 출시 일정은 절대 지연되어서는 안 되며, 모든 부서가 협력하여 계획된 날짜를 반드시 지킬 것을 당부했다. 둘째, 소비자 피드백을 빠르게 수집하고 반영할 수 있는 체계를 구축해야 한다고 지시했다. 이를 위해 고객센터 대응 매뉴얼을 조기에 마련하고, 출시 직후에는 소셜 미디어 채널에서의 소비자 반응을 실시간으로 모니터링할 것을 지시했다.

마지막으로 대표는 베타 테스트 단계에서부터 내부 직원뿐 아니라 실제 고객 그룹을 소규모로 참여시켜 생생한 피드백을 받는 것도 고려해 보라고 제안했다. 이를 통해 예상치 못한 문제를 조기에 발견하고 개선할 수 있으며, 고객들의 기대감을 높이는 효과도 있을 것이라고 덧붙였다.

회의는 각 팀이 준비해야 할 후속 과제를 정리하는 것으로 마무리되었다. 개발팀은 10월 20일까지 베타 테스트를 완료하고 결과를 공유하기로 했으며, 마케팅팀은 사전 예약 이벤트 준비를 10월 15일까지 마무리하기로 했다. 기획팀은 예산 집행 계획을 문서로 정리해 대표에게 보고하기로 합의했다.
"""

In [54]:
result = chain.invoke({"text": text})
print(result)

date='2024-10-01' attendees=['박민수 (개발팀 팀장)', '최지현 (마케팅팀 팀장)', '이영희 (기획팀 팀장)', '김철수 (대표)'] topics=['신제품 개발 진행 상황 및 일정', '출시 전후 마케팅 전략', '마케팅 예산 배정 및 집행 계획', '출시 일정 준수 및 소비자 피드백 체계 구축'] decisions=['개발팀은 10월 20일까지 베타 테스트 완료 및 결과 공유', '마케팅팀은 10월 15일까지 사전 예약 이벤트 준비 완료', '기획팀은 예산 집행 계획 문서화 후 대표에게 보고', '출시 일정 엄수 및 부서 간 협력 강화', '고객 피드백 수집 및 반영 체계 조기 구축', '베타 테스트에 실제 고객 소규모 참여 검토'] next_steps=['개발팀은 UI/UX 개선 및 안정성 테스트 집중', '마케팅팀은 인스타그램, 틱톡 중심의 영상 콘텐츠 및 후기 광고 제작', '기획팀은 추가 예산 승인 절차 준비', '고객센터 대응 매뉴얼 조기 마련 및 소셜 미디어 모니터링 체계 구축', '베타 테스트 결과 분석 및 개선 사항 도출']


In [56]:
result.date

'2024-10-01'

In [57]:
print(type(result))

<class '__main__.MeetingSummary'>


In [58]:
result.model_dump()

{'date': '2024-10-01',
 'attendees': ['박민수 (개발팀 팀장)', '최지현 (마케팅팀 팀장)', '이영희 (기획팀 팀장)', '김철수 (대표)'],
 'topics': ['신제품 개발 진행 상황 및 일정',
  '출시 전후 마케팅 전략',
  '마케팅 예산 배정 및 집행 계획',
  '출시 일정 준수 및 소비자 피드백 체계 구축'],
 'decisions': ['개발팀은 10월 20일까지 베타 테스트 완료 및 결과 공유',
  '마케팅팀은 10월 15일까지 사전 예약 이벤트 준비 완료',
  '기획팀은 예산 집행 계획 문서화 후 대표에게 보고',
  '출시 일정 엄수 및 부서 간 협력 강화',
  '고객 피드백 수집 및 반영 체계 조기 구축',
  '베타 테스트에 실제 고객 소규모 참여 검토'],
 'next_steps': ['개발팀은 UI/UX 개선 및 안정성 테스트 집중',
  '마케팅팀은 인스타그램, 틱톡 중심의 영상 콘텐츠 및 후기 광고 제작',
  '기획팀은 추가 예산 승인 절차 준비',
  '고객센터 대응 매뉴얼 조기 마련 및 소셜 미디어 모니터링 체계 구축',
  '베타 테스트 결과 분석 및 개선 사항 도출']}

In [59]:
try:
    result = chain.invoke({"text": text})
    print(result)
except ValidationError as e:
    print("ValidationError", e)

date='2024-06' attendees=['박민수 (개발팀 팀장)', '최지현 (마케팅팀 팀장)', '이영희 (기획팀 팀장)', '김철수 (대표)'] topics=['신제품 개발 진행 상황 및 일정', '출시 전후 마케팅 전략', '마케팅 예산 배정 및 집행 계획', '출시 일정 준수 및 소비자 피드백 체계 구축'] decisions=['개발팀은 10월 20일까지 베타 테스트 완료 및 결과 공유', '마케팅팀은 10월 15일까지 사전 예약 이벤트 준비 완료', '기획팀은 예산 집행 계획 문서화 후 대표에게 보고', '출시 일정 엄수 및 부서 간 협력 강화', '고객 피드백 신속 수집 및 반영 체계 마련', '베타 테스트에 실제 고객 소규모 참여 검토'] next_steps=['개발팀은 UI/UX 개선과 안정성 테스트 집중 진행', '마케팅팀은 인스타그램, 틱톡 중심의 영상 콘텐츠 및 후기 광고 제작', '기획팀은 추가 예산 승인 절차 준비', '고객센터 대응 매뉴얼 조기 마련 및 소셜 미디어 모니터링 체계 구축', '베타 테스트 고객 참여 방안 구체화 및 실행 계획 수립']


## 실습
1. 주제부터 정해보기(예: 회의록 요약하기, 리뷰 요약하기, 시나리오 정하기)